In [26]:
#in python , to request an API we import request module
import requests

In [27]:
#initialising base URL which you use for all requests to this API.
api_base = 'https://geo.weather.gc.ca/geomet'

In [15]:
from datetime import datetime

# Get today's date in the format 'YYYY-MM-DD'
today = datetime.today().strftime('%Y-%m-%d')


# Construct the API parameters
api_params = (
    'SERVICE=WMS'
    '&VERSION=1.3.0'
    '&REQUEST=GetFeatureInfo'
    '&BBOX=51,-114,51.5,-113.5'
    '&CRS=EPSG:4326'
    '&WIDTH=10'
    '&HEIGHT=10'
    '&LAYERS=GDPS.ETA_TT'
    '&QUERY_LAYERS=GDPS.ETA_TT'
    '&INFO_FORMAT=application/json'
    '&I=5'
    '&J=5'
    f'&TIME={today}T15:00:00Z'  # Using today's date
)

# Print the result to see the query
print(api_params)

SERVICE=WMS&VERSION=1.3.0&REQUEST=GetFeatureInfo&BBOX=51,-114,51.5,-113.5&CRS=EPSG:4326&WIDTH=10&HEIGHT=10&LAYERS=GDPS.ETA_TT&QUERY_LAYERS=GDPS.ETA_TT&INFO_FORMAT=application/json&I=5&J=5&TIME=2025-03-01T15:00:00Z


In [16]:
url = api_base + '?' + api_params
url

'https://geo.weather.gc.ca/geomet?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetFeatureInfo&BBOX=51,-114,51.5,-113.5&CRS=EPSG:4326&WIDTH=10&HEIGHT=10&LAYERS=GDPS.ETA_TT&QUERY_LAYERS=GDPS.ETA_TT&INFO_FORMAT=application/json&I=5&J=5&TIME=2025-03-01T15:00:00Z'

In [17]:
from datetime import datetime, timedelta

print(datetime.now())
my_time_difference = -7
print(datetime.now() + timedelta(hours=my_time_difference))

2025-03-01 13:59:03.614948
2025-03-01 06:59:03.615116


In [18]:
def round_multiple(num, multiple):

    return ((num + multiple // 2) // multiple) * multiple

In [19]:
def get_api_datetime(time: datetime) -> str:
    """Rounds `time` to the nearest hour multiple of 3, and returns `time`
    formatted to a ISO8601 UTC time string expected by the API."""
    # Convert the time to UTC
    utc_time = datetime.utcfromtimestamp(time.timestamp())
    # Round to an hour the API accepts
    rounded_hour = round_multiple(utc_time.hour, 3)

    # Check if we're going into the next day after rounding
    if rounded_hour >= 24:
        # Round to the start of the next day
        utc_time = utc_time.replace(hour=0, minute=0, second=0) + timedelta(days=1)
    else:
        # Update to the start rounded hour
        utc_time = utc_time.replace(hour=rounded_hour, minute=0, second=0)

    # isoformat gives the ISO8601 string the API expects
    return utc_time.isoformat(timespec='seconds')

get_api_datetime(datetime.now())

'2025-03-01T12:00:00'

In [20]:
def get_api_url(time: datetime, lat: float, lng: float, layers='GDPS.ETA_TT'):
    """Builds the API URL to request data for the given time and location.

    Returns the URL and the time the URL will predict for.

    `layers` gives the type of data to request, the default is to get the air
    temperature prediction.
    """
    pred_time = get_api_datetime(time)

    url = (
    'https://geo.weather.gc.ca/geomet?SERVICE=WMS&VERSION=1.3.0' # Base API URL, common to all requests
    '&REQUEST=GetFeatureInfo' # The request type we want to make, GetFeatureInfo returns weather data
    '&BBOX={lat},{lng},{lat_end},{lng_end}' # The location we want the prediction to be for
    '&CRS=EPSG:4326&WIDTH=10&HEIGHT=10' # Necessary parameters, not important to us
    '&LAYERS={layers}' # The specific type of data we want to be returned
    '&QUERY_LAYERS={layers}' # The specific type of data we want to be returned (necessary duplication)
    '&INFO_FORMAT=application/json' # Return data in JSON format
    '&I=5&J=5' # Necessary parameters, not important to us
    '&TIME={time}Z' # The time the prediction should be for
    )

    return url.format(lat=lat,
                      lng=lng,
                      lat_end=lat+0.5,
                      lng_end=lng+0.5,
                      time=pred_time,
                      layers=layers), pred_time

In [21]:
def get_air_temperature(time: datetime):
    """Returns the JSON response from an air temperature request with the given
    time, as well as the time the prediction was made for."""
    url, pred_time = get_api_url(time, lat=51, lng=-114)
    res = requests.get(url)
    return res.json(), pred_time

In [22]:
import pandas as pd

def get_air_temp_predictions(num=5) -> pd.DataFrame:
    """Returns a Pandas dataframe containing predicted air temperatures,
    with the prediction times as the row labels."""
    air_temps = []
    pred_times = []

    now = datetime.now()
    # Get the next `num` 3 hour intervals
    times = [now + timedelta(hours=3 * i) for i in range(num)]

    for time in times:
        air_temp_json, pred_time_str = get_air_temperature(time)

        # Get the actual prediction value buried in the JSON
        air_temp = air_temp_json['features'][0]['properties']['value']
        air_temps.append(float(air_temp))

        pred_times.append(pred_time_str)

    df = pd.DataFrame({'air_temp': air_temps}, index=pred_times)
    # Parse the prediction time as dates
    df.index = pd.to_datetime(df.index)
    return df

# Change False to True to retrieve fresh predictions
if False or 'air_temp_predictions_gathered' not in globals().keys():
    air_temps_df = get_air_temp_predictions()
    air_temp_predictions_gathered = True

In [25]:
# Update the x-axis values to show based on your time, not UTC
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
x = air_temps_df.index + timedelta(hours=my_time_difference)
y = air_temps_df['air_temp']

# Show the temperatures in increments of 10
y_min = round_multiple(y.min(), 10)
if y_min > y.min():
    y_min -= 10

y_max = round_multiple(y.max(), 10)
if y_max < y.max():
    y_max += 10

p = figure(title='Air temperature forecast', x_axis_type='datetime',
           x_axis_label='Time of day', y_axis_label='Temperature (Celsius)',
           y_range=[y_min, y_max])

p.line(x=x, y=y)
p.scatter(x=x, y=y)

show(p)

In [ ]:
def get_api_capabilities():
    """Writes the response of the API capabilities request to `capabilities.xml`.

    This request returns XML with data on everything the API can do.
    """
    url = 'https://geo.weather.gc.ca/geomet?SERVICE=WMS&REQUEST=GetCapabilities&VERSION=1.3.0'

    res = requests.get(url)

    with open('capabilities.xml', 'w') as f:
        f.write(res.text)

get_api_capabilities()

In [ ]:
url, pred_time = get_api_url(datetime.now(), lat=51, lng=-114, layers='GDPS.ETA_ES')
res = requests.get(url)
res.json()